In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display, clear_output

import yfinance as yf
from finvizfinance.quote import finvizfinance


In [131]:
#print(type(finvitz_data))

In [132]:
def yfinance_func(nome_ticker):
    global dati_storici
    ticker = yf.Ticker(nome_ticker.upper())
    dati_storici = ticker.history(period="max")  # dati periodo massimo disponibile
    
    if len(dati_storici) > 0:
        display(len(dati_storici))
    else:
        display('Titolo inesistente o delistato')
        
        
        
        
# fondamentali da yahoo finance 

#stock = yf.Ticker(nome_ticker)
#
#mcap = stock.info.get('marketCap')
#outstanding = stock.info.get('outStanding')
#shares_float = stock.info.get('floatShares')
#short_float = stock.info.get('shortPercentOfFloat')*100
        

In [133]:
# CARICO I DATI FONDAMENTALI DA FINVITZ

def finvitz_func(nome_ticker):
    
    
    #global finvitz_data
    
    tentativi = 1
    while tentativi < 5:

        try:
            
            # fondamentali da finviz
            
            stock = finvizfinance(nome_ticker)
            finvitz_data = stock.ticker_fundament()
            
            mcap = finvitz_data.get('Market Cap')
            shares_float = finvitz_data.get('Shs Float')
            insider = finvitz_data.get('Insider Own')
            institutional = finvitz_data.get('Inst Own')
            short_float = finvitz_data.get('Short Float')
            
            #print(f"accesso ai dati riuscito al tentativo {tentativi}")
            print('\n\n',nome_ticker,'\n')
            print(f'Market cap: {mcap}')
            #print(f'Outstanding: {outstanding}')
            print(f'float: {shares_float}')
            print(f'short float: {short_float}')
            print(f'insider ownership: {insider}')
            print(f'institutional ownership: {institutional} \n\n')
            
            break
        
        except: #Exception as e:
            
            print(f'tentativo {tentativi}/5 di accesso ai dati finvitz fallito')
            tentativi += 1
            #display(e)
        
        
    if tentativi == 5: 
        print("tentativi esauriti")
        
        
        
        
        
    

In [103]:
def elaborazione(b):
    
    global trans,dati_storici_ADJ 
    
    # APPORTO dei CORRETTIVI al DF dati_storici  ORIGINALE
    global dati_storici

    dati_storici = dati_storici.reset_index()
    dati_storici['Data'] = dati_storici['Date'].dt.date
    dati_storici.drop('Date',inplace=True,axis=1)
    dati_storici.rename(columns={'Data':'Date'},inplace=True)
    colonna_da_spostare = dati_storici.pop('Date')
    dati_storici.insert(0,'Date',colonna_da_spostare)
    
    # converto in formato datetime
    #dati_storici['Date'] = pd.to_datetime(dati_storici['Date'])
    
    
    
    
    
    
    
    # creo un df transitorio dove inserire i risultati del calolo dello split factor:

    trans = pd.DataFrame(index=range(0,len(dati_storici)))
    trans['split_factor'] = 0



    # inizio con il calcolo:
    
    split_factor = 0
    
    
    for a in range((len(dati_storici)-1),-1,-1):
                   if dati_storici['Stock Splits'].iloc[a] > 0:
                               #display(dati_storici['Stock Splits'].iloc[a])
                               if split_factor == 0:
                                   split_factor = dati_storici['Stock Splits'].iloc[a]
                                   trans.loc[a,'split_factor'] = split_factor
                                   display(split_factor)
                               else: 
                                   split_factor = split_factor*dati_storici['Stock Splits'].iloc[a]
                                   
                   
                               #break
                    
                   else:
                               if dati_storici['Stock Splits'].iloc[a] == 0 and split_factor > 0:
                                   trans.loc[a,'split_factor'] = split_factor 
                               else:
                                   trans.loc[a,'split_factor'] = 1
                                
                            
    
    
    
    
    dati_storici['Gap %'] = ((dati_storici['Open']*100)/dati_storici['Close'].shift(1))-100

    colonna_da_spostare = dati_storici.pop('Gap %')
    dati_storici.insert(1,'Gap %',colonna_da_spostare)
    
    dati_storici['Max % UP'] = ((dati_storici['High']*100)/(dati_storici['Open']))-100
    dati_storici['Max % DOWN'] = ((dati_storici['Low']*100)/(dati_storici['Open']))-100
    dati_storici['Open to Close %'] = ((dati_storici['Close']*100)/(dati_storici['Open']))-100
    dati_storici['Chiusura'] = dati_storici.apply(lambda x: 'RED' if x['Open to Close %']<0 \
                                                  else 'GREEN' if x['Open to Close %']>0 else '=open', axis=1)
    
    dati_storici = dati_storici.round(3)
    
    
    # creo una copia di dati_storici per mantenere inalterati i prezzi corretti che serviranno x il grafico
    dati_storici_ADJ = dati_storici.copy()
    
    

In [62]:
# richiamo le funzioni per generare la tabella, il grafico, e la gestione tramita tendina del grafico

In [128]:
def ricerca_gaps(gap_perc_A,gap_perc_B,volume):
    

    #print(nome_ticker,'\n')
    #print(f'Market cap: {mcap}')
    ##print(f'Outstanding: {outstanding}')
    #print(f'float: {shares_float}')
    #print(f'short float: {short_float}')
    #print(f'insider ownership: {insider}')
    #print(f'institutional ownership: {institutional} \n\n')
    print(f'Storico GAPs >= {gap_perc_A}%')
    print(f'con Volume minimo >= {volume/1000000:.2f} mln \n')
    
    global gaps
    gaps = dati_storici[(dati_storici['Gap %']>=gap_perc_A)&\
                        (dati_storici['Gap %']<=gap_perc_B)&\
                        (dati_storici['Volume']>=volume)].iloc[:,[0,1,2,3,4,5,9,10,11,-1,6]]
    
    
    # rettifico al valore originale i dati di ogni riga del df gaps:
    
    if gaps.shape[0]>0:
        
        gaps['Open'] = gaps.apply(lambda x: x['Open']*trans.loc[x.name,'split_factor'],axis=1)
        gaps['High'] = gaps.apply(lambda x: x['High']*trans.loc[x.name,'split_factor'],axis=1)
        gaps['Low'] = gaps.apply(lambda x: x['Low']*trans.loc[x.name,'split_factor'],axis=1)
        gaps['Close'] = gaps.apply(lambda x: x['Close']*trans.loc[x.name,'split_factor'],axis=1)
        gaps['Volume'] = gaps.apply(lambda x: x['Volume']/trans.loc[x.name,'split_factor'],axis=1)
    
        
        # Effettuo adesso una copia del df gaps - solamente per poterla visualizzare
    
        display_gaps = gaps.copy()
        
        
        display_gaps['Volume'] = display_gaps.apply(lambda x: f"{x['Volume']:,.0f}".replace(',','.'),axis=1)
        
        display_gaps = display_gaps.round(2)
        
        
        display_gaps[['Gap %', 'Open', 'High', 'Low', 'Close', 'Max % UP',
           'Max % DOWN', 'Open to Close %']]=\
        display_gaps[['Gap %', 'Open', 'High', 'Low', 'Close', 'Max % UP',
           'Max % DOWN', 'Open to Close %']].astype(str).apply(lambda x: x.str.replace('.',',',regex=False))
        
        
        
        display_gaps.reset_index(drop=True,inplace=True)
        display_gaps.index = display_gaps.index+1
    
    
        display(display_gaps)
        #display(gaps)
        
        
    
        return(gaps)
        
        
        
    else: 
        print(f' il titolo {nome_ticker.value} non ha nessun gap superiore o uguale al {gap_perc_A}%')
        
        
        

In [124]:
def visual_gap(n_gap):
    
    #global gaps
    global dati_storici_ADJ
    
    finestra_daily = 100

    elementi_da_inizio_df = gaps.index[n_gap]
    if elementi_da_inizio_df > round(finestra_daily/2):
        finestra_A = round(finestra_daily/2)
    else:
        finestra_A = elementi_da_inizio_df
        
    #print(finestra_A )
    
        
    elementi_da_fine_df = (dati_storici_ADJ.shape[0]-1)-elementi_da_inizio_df
    if elementi_da_fine_df > round(finestra_daily/2):
        finestra_B = round(finestra_daily/2)
    else:
        finestra_B = elementi_da_fine_df
    #print(finestra_B )    
    
    
    
    
    
    df = dati_storici_ADJ.iloc[gaps.index[n_gap]-(finestra_A)\
                               :gaps.index[n_gap]+(finestra_B),:].copy()
    
    
    # Crea una colonna con il testo da mostrare nel tooltip
    df['hover_text'] = (
        "Data: " + df['Date'].astype(str) + "<br>" +
        "Open: " + df['Open'].astype(str) + "<br>" +
        "High: " + df['High'].astype(str) + "<br>" +
        "Low: " + df['Low'].astype(str) + "<br>" +
        "Close: " + df['Close'].astype(str) + "<br>" +
        "Gap %: " + df['Gap %'].astype(str) + "<br>"
    )
    
    df['volume_text'] = df.apply(lambda x: f"{x['Volume']:,.0f}".replace(",","."),axis=1)
    df['volume_text'] = ("Volume: "+df['volume_text'].astype(str))
    
    
    
    # Crea il grafico con il testo personalizzato
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        row_heights=[0.8, 0.2],
        vertical_spacing= 0.15
    )
    
    # Aggiunta delle candele
    fig.add_trace(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name="Daily",
            hovertext=df['hover_text'],  # Assegna il testo personalizzato
            hoverinfo="text"  # Specifica di mostrare solo il testo personalizzato
        ),
        row=1, col=1
    )
    
    # Aggiunta del volume
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df['Volume'],
            name="Volume",
            hovertext=df['volume_text'],
            hoverinfo="text",
            marker_color='blue',
            opacity=0.6
        ),
        row=2, col=1
    )
    
    
    
    if finestra_A >= 5:
        finestra_visione_A = 5 
    else:
        finestra_visione_A = finestra_A-1   
    
    if finestra_B >= 5:
        finestra_visione_B = 5 
    else:
        finestra_visione_B = finestra_B-1
    
    
    
    
    # Layout
    fig.update_layout(
        title=f"{nome_ticker.value} -  Grafico Gap del {gaps.iloc[n_gap,0]}",
        #xaxis_title="Date",
        yaxis_title="Prezzo",
        xaxis = {'range': (df['Date'].loc[gaps.index[n_gap]-finestra_visione_A], 
                           df['Date'].loc[gaps.index[n_gap]+finestra_visione_B]),
                'rangeslider': {'visible': True} },
        #xaxis_rangeslider_visible=True,
        xaxis_rangeslider = {'thickness':0.08,'visible':True},
        template="plotly_white",
        width=800,
        height=600,
        
        
        
        shapes = [
        {
            # Sfondo colorato al centro del range
            'type': "rect" ,
            'xref': "x",  # Si riferisce all'asse X
            'yref': "paper",  # Si riferisce all'intero grafico (da 0 a 1)
            'x0': df['Date'].loc[gaps.index[n_gap]],  # Inizio dello sfondo
            'x1': df['Date'].loc[gaps.index[n_gap]- 1],  # Fine dello sfondo
            'y0': 0.80,  # Parte bassa del grafico
            'y1': 0.40,  # Parte alta del grafico
            'fillcolor': 'Orange',  # Colore dello sfondo (rosso trasparente)
            'opacity':0.1,
            'layer': "below",  # Metti lo sfondo sotto i dati
            'line_width': 0  # Nessun bordo
        }
    ]
    
        
           
    )
    
    
    ## Riposiziona lo slider manualmente per metterlo sotto i volumi
    #fig.update_layout(
    #    xaxis_rangeslider=dict(
    #        visible=True,
    #        yanchor="bottom",  # Posiziona lo slider sotto
    #        thickness=0.1      # Spessore dello slider
    #    ),
    #    height=700  # Aumenta l'altezza per fare spazio allo slider
    #)
    
    
    fig.show()


In [106]:

def richiamo_grafico():
    
    # Crea una tendina con un'opzione iniziale vuota e numeri da 1 in poi
    n_gap_selector = widgets.Dropdown(
        options=[("", None)] + [(f"Gap {i+1}", i) for i in range(len(gaps))],  # Opzione vuota + opzioni numerate
        description='Visualizzazione GAP:',
        style={'description_width': '150px'},
        value=None,  # La selezione iniziale è vuota
        layout=widgets.Layout(width='170px',margin='50px 0px 30px 0px')  # Riduce la larghezza della tendina
    )
    
    # Output per mostrare il grafico
    output = widgets.Output()
    
    # Funzione per gestire la selezione dalla tendina
    def on_gap_change(change):
        if change.new is not None:  # Esegui solo se un gap valido è selezionato
            output.clear_output(wait=True)  # Pulisce l'output precedente
            with output:
                visual_gap(change.new)      # Chiama la funzione passando il valore selezionato
    
    # Collegare la funzione al widget
    n_gap_selector.observe(on_gap_change, names='value')
    
    # Visualizza la tendina e l'output
    display(widgets.VBox([n_gap_selector, output]))


In [107]:
def attivazione_tabella_grafici(b):

    tasto_premuto = 0
    
    # Variabile globale per salvare il DataFrame risultante
    df_risultante = None
    
    # Funzione per formattare i numeri in milioni
    def format_volume(value):
        return f"{value:.1f} mln".replace(".", ",")
    
    # Slider per selezionare le percentuali di Gap (ora interi e inizio da 0)
    gap_slider = widgets.IntRangeSlider(
        value=[0, 500],  # Inizio con i cursori su min e max
        min=0,  # Valore minimo aggiornato
        max=500,
        step=5,
        description="Gap %",
        layout=widgets.Layout(width="50%")
    )
    
    # Slider per selezionare il volume minimo, con step di 0.5 milioni
    volume_slider = widgets.FloatSlider(
        value=0.5,  # Valore iniziale
        min=0.5,  # Valore minimo
        max=100.0,  # Valore massimo
        step=0.5,  # Step incrementale
        layout=widgets.Layout(width="31%"),
        readout=False  # Disabilita il valore numerico predefinito
    )
    
    # Etichetta statica a sinistra dello slider
    volume_slider_label = widgets.Label("Volume minimo:")
    
    # Etichetta dinamica a destra dello slider
    volume_slider_value = widgets.Label(format_volume(volume_slider.value))
    
    # Funzione per aggiornare il valore formattato a destra dello slider
    def update_volume_description(change):
        volume_slider_value.value = format_volume(volume_slider.value)
    
    # Osserva i cambiamenti di valore per aggiornare l'etichetta dinamica
    volume_slider.observe(update_volume_description, names="value")
    
    # Pulsante per avviare la ricerca
    search_button = widgets.Button(
        description="Cerca GAPs",
        button_style="success",  # Colore verde
        tooltip="Avvia la ricerca dei gap",
        icon="search",
        layout=widgets.Layout(margin="30px 0px 30px 0px")  # Spazio sotto il pulsante
    )
    
    # Output per visualizzare i risultati
    output = widgets.Output()
    
    # Funzione per gestire il click del pulsante
    def on_search_click(b):
        global tasto_premuto
        tasto_premuto = 1
        with output:
            clear_output(wait=True)
            gap_perc_A, gap_perc_B = gap_slider.value
            volume = volume_slider.value * 1_000_000  # Converti da milioni a numero reale
            
            df_gaps = ricerca_gaps(gap_perc_A, gap_perc_B, volume)  # visualizza il risultato
            
            if isinstance(df_gaps,pd.DataFrame):
            
                if tasto_premuto == 1:
                    richiamo_grafico()
            
            
            
    
    # Collegamento dell'evento click
    search_button.on_click(on_search_click)
    
    # Visualizzazione degli elementi
    display(widgets.VBox([
        gap_slider,
        widgets.HBox([volume_slider_label, volume_slider, volume_slider_value]),  # Slider e descrizioni
        search_button,
        output
    ]))
    

In [134]:

## INSERISCO il TICKER
#
#
#nome_ticker = widgets.Text(
#    value='',
#    placeholder='Inserisci il nome del ticker',
#    description='nome Ticker:',
#    disabled=False)
#
#button = widgets.Button(description="Carica Dati")
#
#
#def premi_tasto(b):
#    
#    print('loading...')
#    #clear_output(wait=True)
#    
#    finvitz_func(nome_ticker.value)
#    yfinance_func(b)
#    
#    elaborazione(b)
#    
#    attivazione_tabella_grafici(b)
#    
#
#
#    
#    
#
#    
#button.on_click(premi_tasto)
#    
#display(nome_ticker, button)  


# INSERISCO il TICKER
global bome_ticker
nome_ticker = widgets.Text(
    value='',
    placeholder='Inserisci il nome del ticker',
    description='nome Ticker:',
    disabled=False
)

button = widgets.Button(description="Carica Dati")

# Output per visualizzare i messaggi e i risultati
output = widgets.Output()

def premi_tasto(b):
    with output:
        # Cancella l'output precedente
        output.clear_output(wait=True)
        print("Loading...")
        output.clear_output(wait=True)
        
        
        try:
            # Verifica se è stato inserito un ticker
            if not nome_ticker.value.strip():
                print("Inserire un nome valido per il ticker.")
                return
            
            # Esegui le funzioni principali
            finvitz_func(nome_ticker.value)  # Passa il valore del ticker
            yfinance_func(nome_ticker.value)  # Modifica per passare il ticker
            elaborazione(b)  # Se non usa parametri, rimuovi `b`
            attivazione_tabella_grafici(b)  # Se non usa parametri, rimuovi `b`
            #print("Elaborazione completata.")
        
        except Exception as e:
            print(f"Errore durante l'elaborazione: {e}")

# Collega il pulsante al callback
button.on_click(premi_tasto)

# Visualizza i widget
display(widgets.VBox([nome_ticker, button, output]))

    


In [102]:
# qui sotto gestisco con sliders la creazione della tabella gaps

In [ ]:
#voila /Users/ninni/Desktop/Giammò/DATI/GAPPERs_WEB_trial.ipynb

In [97]:
#import ipywidgets as widgets
#from IPython.display import display
#
## Crea alcuni widget (etichette, slider, bottoni)
#label1 = widgets.Label("Riga 1 colonna 1")
#label2 = widgets.Label("Riga 2 colonna 1")
#slider1 = widgets.IntSlider(description="Slider 1")
#
#label3 = widgets.Label("Riga 1 colonna 2")
#label4 = widgets.Label("Riga 2 colonna 2")
#slider2 = widgets.IntSlider(description="Slider 2")
#
## Usa VBox per impilare più righe nella stessa colonna
#column1 = widgets.VBox([label1, label2, slider1])
#column2 = widgets.VBox([label3, label4, slider2])
#
## Usa HBox per affiancare le due colonne
#display(widgets.HBox([column1, column2]))


In [98]:
#import pandas as pd
#import ipywidgets as widgets
#from IPython.display import display
#
## Crea due DataFrame di esempio
#df1 = pd.DataFrame({
#    'A': [1, 2, 3],
#    'B': [4, 5, 6]
#})
#
#df2 = pd.DataFrame({
#    'X': [10, 20, 30],
#    'Y': [40, 50, 60]
#})
#
## Crea i widget per visualizzare i DataFrame
#output1 = widgets.Output()
#output2 = widgets.Output()
#
## Visualizza i DataFrame all'interno dei widget Output
#with output1:
#    display(df1)
#
#with output2:
#    display(df2)
#
## Usa HBox per affiancare i due DataFrame
#display(widgets.HBox([output1, output2]))
#